# PsyNer project

## Installing all necessary libraries and functionalities

In [7]:
pip install spacy -q

Note: you may need to restart the kernel to use updated packages.


In [8]:
!python -m spacy download en_core_web_sm en_core_web_lg -q

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Generalized spaCy models

In [9]:
import spacy

In [18]:
test="After a decades-long pause, psychedelics are again being intensely investigated for treating a wide range of neuropsychiatric ailments including depression, anxiety, addiction, post-traumatic stress disorder, anorexia, and chronic pain syndromes. The classic serotonergic psychedelics psilocybin and lysergic acid diethylamide and nonclassic psychedelics 3,4-methylenedioxymethamphetamine and ketamine are increasingly appreciated as neuroplastogens given their potential to fundamentally alter mood and behavior well beyond the time window of measurable exposure. Imaging studies with psychedelics are also helping advance our understanding of neural networks and connectomics. This resurgence in psychedelic science and psychedelic-assisted therapy has potential significance for the fields of neurosurgery and neuro-oncology and their diverse and challenging patients, many of whom continue to have mental health issues and poor quality of life despite receiving state-of-the-art care. In this study, we review recent and ongoing clinical trials, the set and setting model of psychedelic-assisted therapy, potential risks and adverse events, proposed mechanisms of action, and provide a perspective on how the safe and evidence-based use of psychedelics could potentially benefit many patients, including those with brain tumors, pain syndromes, ruminative disorders, stroke, SAH, TBI, and movement disorders. By leveraging psychedelics' neuroplastic potential to rehabilitate the mind and brain, novel treatments may be possible for many of these patient populations, in some instances working synergistically with current treatments and in some using subpsychedelic doses that do not require mind-altering effects for efficacy. This review aims to encourage broader multidisciplinary collaboration across the neurosciences to explore and help realize the transdiagnostic healing potential of psychedelics."

In [19]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(test)
for ent in doc.ents:
    print (ent.text, ent.label_)

decades DATE
3,4-methylenedioxymethamphetamine QUANTITY
SAH ORG
TBI ORG


In [20]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(test)
for ent in doc.ents:
    print (ent.text, ent.label_)

decades DATE
3,4 CARDINAL
neuroplastogens PERSON
connectomics WORK_OF_ART
SAH ORG


We can clearly see that neither small nor large spacy models was not able to catch any entities from the provided abstract correctly. It is not surprising since these are very generalized english models and therefore they are not traind on a specific set of biological data. Most likely this model has never seen anything like this abstract before. Moreover, as we can see below, our model doesn't even posess the labels that we want. Thus, we will explore some other models and the ways to customize a model according to our needs.

In [27]:
for pipe_name in nlp.pipe_names:
    if pipe_name == 'ner':
        component = nlp.get_pipe(pipe_name)
        if hasattr(component, "labels"):
            print(component.labels)

('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')


CARDINAL: Numerals that do not fall under another type
DATE: Absolute or relative dates or periods
EVENT: Named hurricanes, battles, wars, sports events, etc.
FAC: Buildings, airports, highways, bridges, etc.
GPE: Countries, cities, states
LANGUAGE: Any named language
LAW: Named documents made into laws
LOC: Non-GPE locations, mountain ranges, bodies of water
MONEY: Monetary values, including unit
NORP: Nationalities or religious or political groups
ORDINAL: "first", "second", etc.
ORG: Companies, agencies, institutions, etc.
PERCENT: Percentage, including "%"
PERSON: People, including fictional
PRODUCT: Objects, vehicles, foods, etc. (not services)
QUANTITY: Measurements, as of weight or distance
TIME: Times smaller than a day
WORK_OF_ART: Titles of books, songs, etc.

## Creating a custom model

In [38]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import json
import os
import random
from tqdm import tqdm

%reload_ext autoreload
%autoreload 2
import datahelper as dt

In [35]:
TRAIN_DATA = dt.load_data("train_data.json")

In [41]:
nlp.pipe_names

['entity_ruler']

In [ ]:
nlp.add_pipe("ner", name="psy_ner",before="")